In [103]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as sm
from sklearn.metrics import mean_squared_error,r2_score,explained_variance_score
from datetime import date
import matplotlib.pyplot as plt



def dateFromFloat(date_Float:float):
    day= date_Float % 100
    month = (date_Float // 100)%100 
    year = date_Float // 10000
    return date(year,month,day)





In [120]:
#Load the data
file0 = "/Users/apple/Workspace/intelliJ/jockey/regression.csv"
dataset0 = pd.read_csv(file0)
raceClass=5
dataset_train=dataset0[(dataset0['raceClass']==raceClass) & (dataset0['rateDate']!=20190106)]
dataset_test=dataset0[(dataset0['raceClass']==raceClass) & (dataset0['rateDate']==20190106)]



In [117]:
features=['distance','draw','addedWeight','declaredHorseWeight','horse_winPer',
                 'horse_winCount','jockey_winPer','jockey_winCount','horse_newDistance','horse_newHorse']
target=['lbw','finishTime']
#regression model of lbw or finishTime
X_train=dataset_train[features]
y_train=dataset_train[target] #lbw

X_test=dataset_test[features]
y_test=dataset_test[target] #lbw


reg = LinearRegression()
reg.fit(X_train,y_train)
y_test_predicted=reg.predict(X_test)
plt.scatter(y_test, y_test_predicted,  color='black')
    
plt.show()
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_test_predicted))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test, y_test_predicted))

explained_variance_score(y_test, y_test_predicted)



ValueError: Found array with 0 sample(s) (shape=(0, 10)) while a minimum of 1 is required.

In [119]:
df = pd.DataFrame(data=y_test_predicted, index=y_test.index,columns=['lbw_predicted','finishTime_predicted'])
dataset_test_toshow=dataset_test[['raceSeqOfDate','rateDate','raceClass','horseId','place','winOdds','lbw','finishTime']]
dataset_test_withPredicted=dataset_test_toshow.merge(df,how='inner',left_index=True,right_index=True)
dataset_test_withPredicted = dataset_test_withPredicted.sort_values(['raceSeqOfDate','finishTime_predicted'])
dataset_test_withPredicted

ValueError: Shape of passed values is (2, 14), indices imply (2, 0)

In [ ]:
frames = [X_test, y_test]
result = pd.concat(frames,axis=0, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True)
result